# Testing Implementation

The goal of this notebook is to mock one response generation and single grading to undestand the results and how to parser everything

In [1]:
from dmcr.models import GenericInstructModelHF
from pathlib import Path
from utils.set_random_seed import set_random_seed
import os
import re

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

set_random_seed(42)
root = "../.."


/home/caio.rhoden/miniconda3/envs/nq_dl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
llm = GenericInstructModelHF(path=f"{root}/models/Llama-3.2-3B-Instruct")
configs = {
    "temperature": 0.7,
    "max_new_tokens": 10,
    "top_p": 0.9
}

question = "when did veterans day start being called veterans day?"
instruction = "Answer the question in 5 tokens or less."
prompt = f"{question}\n"
response = llm.run(prompt=prompt, instruction=instruction, config_params=configs)[0]["generated_text"]

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.01it/s]
Device set to use cuda:0


In [5]:
response

"1944, but it wasn't officially called Veterans"

In [6]:
evaluation_prompt = f""""
[System] 
Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to a question displayed below. Your evaluation should consider factors such as relevance and accuracy. Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, please rate the response on a scale of 1 to 10 by strictly following this format: "[[rating]]", for example: "Rating: [[5]]".  
[Question] 
{question}  
[The Start of Assistant’s Answer] 
{response}
[The End of Assistant’s Answer]
"""
evaluation_prompt

'"\n[System] \nPlease act as an impartial judge and evaluate the quality of the response provided by an AI assistant to a question displayed below. Your evaluation should consider factors such as relevance and accuracy. Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, please rate the response on a scale of 1 to 10 by strictly following this format: "[[rating]]", for example: "Rating: [[5]]".  \n[Question] \nwhen did veterans day start being called veterans day?  \n[The Start of Assistant’s Answer] \n1944, but it wasn\'t officially called Veterans\n[The End of Assistant’s Answer]\n'

In [ ]:
configs = {
    "temperature": 0.5,
    "top_p": 0.9,
    "num_return_sequences": 5,
}
grades = llm.run(prompt=evaluation_prompt, instruction="", config_params=configs)

Device set to use cuda:0


In [8]:
grades

[{'generated_text': "In evaluating the quality of the response provided by the AI assistant, I consider factors such as relevance, accuracy, completeness, and clarity. The response should directly address the question asked and provide a clear and concise answer. In this case, the AI assistant's response attempts to answer the question about when Veterans Day started being called Veterans Day, but it falls short in several areas.\n\nThe response starts by providing a year (1944), which is accurate in the sense that World War II ended in 1945, but it was in 1954 that the U.S. government officially declared November 11 as Veterans Day. However, the response does not provide the correct information about when it was officially called Veterans Day, which is a significant omission. Furthermore, the response is incomplete, as it does not mention the significance of the Armistice Day holiday that was celebrated on November 11 prior to 1954.\n\nRating: [[4]]"},
 {'generated_text': 'I\'ll evalu

In [9]:
def calculate_rating_mean(data):
    """
    Calculate the mean rating from generated texts containing 'Rating: [[number]]' patterns.
    
    Args:
        data: List of dictionaries with 'generated_text' keys
        
    Returns:
        Mean rating (float), or None if no ratings found
    """
    ratings = []
    
    for item in data:
        text = item['generated_text']
        # Search for rating pattern
        match = re.search(r'Rating: \[\[(\d+)\]\]', text)
        if match:
            ratings.append(int(match.group(1)))
    
    if not ratings:
        return None
    
    return sum(ratings) / len(ratings)

In [10]:
calculate_rating_mean(grades)

5.2